# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Сформировать обучающую выборку
- Спроектировать схему валидации с учетом специфики задачи
- Обосновать выбор способа валидации


# Preprocessing text_data

# Импорт библиотек

In [ ]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import pyarrow.parquet as pq
import dill
import re 


from IPython.display import Image

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import seaborn as sns

# USE THIS STYLE
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# 
# OR THIS STYLE
import aquarel

import warnings

warnings.filterwarnings("ignore")

theme = aquarel.load_theme("arctic_light")
theme.set_font(family="serif")
theme.apply()

# Сделаем автоподгрузку всех изменений при перепрогонке ячейки
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from utils import plt_distr, custom_pallete

# Данные

Путь до данных

In [ ]:
data_path = "../../data_closed/"

info_path = "../../data/dress_chars/"

## text data

### Чтение данных

In [ ]:
df_text_pq = pq.read_table(data_path + "text_data_69020_final.parquet")

Возьмем только первый батч, чтобы посмотреть содержание таблицы 

> опытным путем было выяснено, что вся таблица в формате pandas не умещается в ОЗУ 

## Обработка цвета товара

In [ ]:
popular_colors = pd.read_csv(info_path + "colors.csv")
popular_colors["type"] = popular_colors["type"].apply(str.lower)
popular_colors = popular_colors["type"].values
popular_colors

In [ ]:
colors_dict = {color: i + 1 for i, color in enumerate(popular_colors)}
colors_dict["другой"] = len(popular_colors) + 1

colors_dict[""] = len(popular_colors) + 10
colors_dict["[]"] = len(popular_colors) + 10
colors_dict["['']"] = len(popular_colors) + 10
colors_dict['[""]'] = len(popular_colors) + 10
colors_dict

In [ ]:
def which_color(x: str):
    # using try\except
    # for case if x is not iterable
    # or has other type
    try:
        for c in popular_colors:
            if int(c in x):
                return colors_dict[c]

        return colors_dict["другой"]

    except:
        return colors_dict["другой"]


df_colors = pd.DataFrame([], columns=["nm_id", "color"])

for batch in df_text_pq.to_batches():

    tmp = batch.to_pandas()[["nm_id", "colornames"]]
    tmp["colornames"] = tmp["colornames"].astype(str).apply(which_color).astype(int)
    tmp = tmp.rename(
        columns={
            "colornames": "color",
        }
    )

    df_colors = pd.concat([df_colors, tmp], axis=0)

In [ ]:
df_colors

Сохраним в бинарник

In [ ]:
with open(data_path + "items_colors.dill", "wb") as f:
    dill.dump(df_colors, f)

## Обработка characteristics

Характеристики представлены в виде словаря, в котором нас интересуют поля `'charcName'` и `'charcValues'`. 

Нужно получить пары
['charcName': 'charcValues']

In [ ]:
# Словарь для парсинга колонки characteristics
chars_dict = {
    "длина юбки/платья": "length",
    "длина юбки\\платья": "length",
    "модель платья": "model",
    "назначение": "purpose",
    "покрой": "cut",
    "рисунок": "pattern",
    "тип карманов": "pocket",
    "тип ростовки": "height",
    "тип рукава": "sleeve",
    "состав": "material",
    "вид застежки": "closure",
    "вид застёжки": "closure",
    "вырез горловины": "neckline",
    "страна производства": "country",
}

In [ ]:
# ---------------------------------------------------
# Словари для проверки подстрок и составления таблицы
# ---------------------------------------------------


# Застежка
closure_dict = {
    "без застежки": ["без", "нет"],
    "молния": ["молни"],
    "пуговица": ["пуговиц"],
    "завязка": ["завязк"],
    "пояс": ["пояс"],
    "шнуровка": ["шнур"],
    "резинка": ["резин"],
}

# Рисунок
pattern_dict = {
    "абстракция": ["абстрак"],
    "без рисунка": ["без", "однотон", "нет"],
    "горох": ["горох", "горош"],
    "клетка": ["клет"],
    "леопардовый": ["леопард"],
    "полоска": ["полос"],
    "фигуры": ["фигур", "геометр"],
    "цветы": ["цвет", "растен"],
}

# Назначение
purpose_dict = {
    "большие размеры": ["больш"],
    "вечернее": ["вечер"],
    "выпускной": ["выпуск"],
    "беремен": ["для беременн", "будущие мамы", "род"],
    "кормления": ["корм"],
    "крещения": ["крещ"],
    "домашнее": ["дом"],
    "повседневная": ["повседнев"],
    "свадьба": ["свадьб"],
    "пляж": ["пляж"],
    "новый год": ["новый"],
    "школа": ["школ"],
    "спорт": ["спорт"],
    "офис": ["офис"],
}

# Карманы
pocket_dict = {
    "в_шве": ["в шв", "бок"],
    "без_карманов": ["без", "нет"],
    "прорезные": ["прорез"],
    "потайные": ["тайн"],
    "накладные": ["наклад"],
}

# Рукава
sleeve_dict = {
    "без рукавов": ["без", "нет"],
    "длинные": ["дли"],
    "короткие": ["кор"],
    "3/4": ["3/4", "3\\4", "34", "3", "4"],
    "7/8": ["7/8", "7\\8", "78", "7", "8"],
}

# Длина
length_dict = {
    "миди": [
        "миди",
        "серед",
        "10",
        "11",
        "ниже",
        "по",
    ],
    "макси": [
        "макси",
        "длин",
        "12",
        "13",
        "14",
        "15",
        "16",
        "в пол",
        "пол",
    ],
    "мини": [
        "мини",
        "кор",
        "9",
        "8",
        "до",
        "выше",
        "корот",
    ],
}


# Модель
model_dict = {
    "футляр": ["футл"],
    "рубашка": ["рубаш"],
    "открытое": ["откр"],
    "запах": ["запах"],
    "прямое": ["прям"],
    "кожаное": ["кожа"],
    "свадьба": ["свад"],
    "лапша": ["лап"],
    "вязаное": ["вяз"],
    "комбинация": ["комб"],
    "футболка": ["футб"],
    "водолазка": ["водо"],
    "бохо": ["бохо"],
    "сарафан": ["сараф"],
    "пиджак": ["пидж"],
    "трапеция": ["трапец"],
    "мини": ["мини"],
    "макси": ["макси"],
    "миди": ["миди"],
    "свободное": ["свобод"],
    "а-силуэт": [
        "а-силуэт",
        "а- силуэт",
        "а -силуэт",
        "а силуэт",
        "асилуэт",
        "a-силуэт",
        "a- силуэт",
        "a -силуэт",
        "a силуэт",
        "aсилуэт",
    ],
    "туника": ["тун"],
    "приталеное": ["тален"],
    "поло": ["поло"],
    "парео": ["парео"],
}


# Покрой
cut_dict = {
    "асимметричный": ["асимметр"],
    "приталенный": ["притален"],
    "рубашечный": ["рубаш"],
    "свободный": ["свободн"],
    "укороченный": ["укороч"],
    "удлиненный": ["удлин"],
    "облегающий": ["облега"],
    "полуприлегающий": ["полуприлег"],
    "прямой": ["прям"],
    "а-силуэт": [
        "а-силуэт",
        "а силуэт",
        "а- силуэт",
        "асилуэт",
        "a-силуэт",
        "a силуэт",
        "a- силуэт",
        "aсилуэт",
    ],
    "оверсайз": ["овер", "over"],
    "трапеция": ["трапец"],
    "длинный": ["длин"],
}

# Тип ростовки
height_dict = {
    "для высоких": [
        "1,7",
        "1,8",
        "1,9",
        "2,0",
        "17",
        "18",
        "19",
        "20",
        "для выс",
        "длявыс",
    ],
    "для невысоких": [
        "1,2",
        "1,3",
        "1,4",
        "1,5",
        "12",
        "13",
        "14",
        "15",
        "невыс",
        "не выс",
        "для невыс",
        "для не выс",
        "дляневыс",
        "дляне выс",
    ],
    "для среднего роста": [
        "для средн",
        "длясредн",
        "1,6",
        "16",
        "средн",
    ],
    "для всех": [
        "для всех",
        "длявсех",
        "безогр",
        "без огр",
    ],
}

# Материал
material_dict = {
    "акрил": ["акрил"],
    "бамбук": ["бамбук"],
    "вискоза": ["вискоза"],
    "кашемир": ["кашемир"],
    "кожа": ["кожа"],
    "лайкра": ["лайкра"],
    "лен": ["лен"],
    "нейлон": ["нейлон"],
    "полиамид": ["полиамид"],
    "полиэстер": ["полиэстер"],
    "спандекс": ["спандекс"],
    "трикотаж": ["трикотаж"],
    "шерсть": ["шерсть"],
    "шелк": ["шелк"],
    "штапель": ["штапель"],
    "шифон": ["шифон"],
    "хлопок": ["хлопок"],
    "эластан": ["эластан"],
}

# Вырез
neckline_dict = {
    "круглый": ["круг"],
    "классический": ["классич"],
    "стойка": ["стойк"],
    "овал": ["овал"],
    "бретели": ["бретел"],
    "v-образный": ["v"],
    "сердечко": ["сердеч"],
    "американка": ["америк"],
    "фигурный": ["фигур"],
    "u-образный": ["u"],
    "гольф": ["гольф"],
    "хомут": ["хомут"],
    "лодочка": ["лодоч"],
    "отложной": ["отлож"],
    "кармен": ["кармен"],
    "бант": ["бант"],
    "капюшон": ["капюш"],
    "квадратный": ["квадра"],
    "открытый": ["откр", "спина", "плеч"],
    "с горлом": ["с горлом", "горло"],
}

# Страна
country_dict = {
    "Россия": ["россия", "russia"],
    "Беларусь": ["беларусь"],
    "Турция": ["турция"],
    "Франция": ["франция"],
    "Киргизия": ["киргизия"],
    "Китай": ["китай", "china"],
    "Италия": ["италия"],
    "Индия": ["индия"],
    "Бангладеш": ["бангладеш"],
    "Узбекистан": ["узбекистан"],
    "Вьетнам": ["вьетнам"],
    "Гонконг": ["гонконг"],
}

In [ ]:
chars_formating_dicts = {
    "length": length_dict,
    "model": model_dict,
    "purpose": purpose_dict,
    "cut": cut_dict,
    "pattern": pattern_dict,
    "pocket": pocket_dict,
    "height": height_dict,
    "sleeve": sleeve_dict,
    "material": material_dict,
    "closure": closure_dict,
    "neckline": neckline_dict,
    "country": country_dict,
}

In [ ]:
def parse_char_dict(characteristics: dict):
    filtered_chars = {}
    for char in characteristics:
        try:
            filtered_chars[chars_dict[str.lower(char["charcName"])]] = [
                str.lower(x) for x in char["charcValues"]
            ]
        except:
            pass
    return filtered_chars


def get_char_value(characteristics: dict, char: str):
    try:
        return characteristics[char]
    except:
        return ["unknown"]


def format_chars(chars: npt.ArrayLike, char_dict: dict):

    for charType, charValues in char_dict.items():
        if any(v in "".join(chars) for v in charValues):
            return charType

    return "unknown"

In [ ]:
chars_df = pd.DataFrame(
    [], columns=["item_id"] + sorted(list(set(chars_dict.values())))
)


for batch in df_text_pq.to_batches():
    df = batch.to_pandas()[["nm_id", "characteristics"]]

    df = df.rename(columns={"nm_id": "item_id"})

    df["characteristics"] = df["characteristics"].apply(parse_char_dict)

    for char in sorted(list(set(chars_dict.values()))):
        df[char] = df["characteristics"].apply(lambda x: get_char_value(x, char))

    for k, v in chars_formating_dicts.items():
        df[k] = df[k].apply(lambda x: format_chars(x, v))

    df = df.drop(columns="characteristics")

    chars_df = pd.concat([chars_df, df], axis=0)

chars_df

In [ ]:
chars_df["item_id"] = chars_df["item_id"].astype(int)

In [ ]:
# Сохраним в бинарник
with open(data_path + "chars_df.dill", "wb") as f:
    dill.dump(chars_df, f)

In [ ]:
with open(data_path + "chars_df.dill", "rb") as f:
    chars_df = dill.load(f)

## Текст

In [ ]:
def text_fmt(text: str):
    try:
        if text is None or text is np.nan:
            return ""
        return re.sub(r"\s\s+", " ", text).lower()
    except:
        return ""

In [ ]:
df_descrs = pd.DataFrame([], columns=["nm_id", "title", "description"])

for batch in df_text_pq.to_batches():

    tmp = batch.to_pandas()[["nm_id", "title", "description"]]

    tmp["title"] = tmp["title"].apply(text_fmt)
    tmp["description"] = tmp["description"].apply(text_fmt)

    tmp["title_len"] = tmp["title"].apply(len)
    tmp["descr_len"] = tmp["description"].apply(len)

    tmp["title_word_len"] = tmp["title"].apply(lambda x: len(x.split()))
    tmp["descr_word_len"] = tmp["description"].apply(lambda x: len(x.split()))

    df_descrs = pd.concat([df_descrs, tmp], axis=0)

df_descrs

In [ ]:
# Сохраним в бинарник
with open(data_path + "df_descrs.dill", "wb") as f:
    dill.dump(df_descrs, f)

#### MANAGE DESCRIPTIONS

In [ ]:
with open(data_path + "df_descrs.dill", "rb") as f:
    df_descrs = dill.load(f)

In [ ]:
df_descrs

In [ ]:
plot_info = {
    "field": "title_len",
    "title": "Распределение длина текста названия товаров (столбец title_len)",
    "annotation": """
    Пик в на длине 6 символов соответствует названию "ПЛАТЬЕ". 
В остальном в названии часто прописывают главные 
характеристики товара (видимо, чтобы давало большее 
соответствии при поиске через строку), что и увеличивает 
длину title
    """,
    "xlabel": "Длина текста, символы",
    "ylabel": "Плотность",
    "ann_xy": (30, 0.02),
    "xlim": (0, 75),
    "ylim": (0, 0.1),
}

plt_distr(df_descrs, plot_info)

In [ ]:
plot_info = {
    "field": "title_word_len",
    "title": "Распределение длина текста названии товаров (столбец title_word_len)",
    "annotation": """
    Аналогично посимвользому распределению
    """,
    "xlabel": "Длина текста, слова",
    "ylabel": "Плотность",
    "ann_xy": (4.5, 0.2),
    "xlim": (0, 10),
    "ylim": (0, 1),
}

plt_distr(df_descrs, plot_info)

In [ ]:
plot_info = {
    "field": "descr_len",
    "title": "Распределение длина текста описания товаров (столбец description_len)",
    "annotation": """
    Из графика можно увидеть, что для товаров описание 
    очень частно не заполнено. Что нтересно, 
    у распределения в районе 1000 и 2000 символов 
    имеются пики (может заполняют для галочки, 
    либо платформа дает какие-то привелегии 
    за достижение описанием определенной длины)
    """,
    "xlabel": "Длина текста, символы",
    "ylabel": "Плотность",
    "ann_xy": (1.2e3, 2e-4),
    "xlim": (0, 2.5e3),
    "ylim": (0, 1.5e-3),
}

plt_distr(df_descrs, plot_info)

In [ ]:
plot_info = {
    "field": "descr_word_len",
    "title": "Распределение длина текста описания товаров (столбец descr_word_len)",
    "annotation": """
    Аналогично посимвользому распределению
    """,
    "xlabel": "Длина текста, слова",
    "ylabel": "Плотность",
    "ann_xy": (200, 2e-3),
    "xlim": (0, 600),
    "ylim": (0, 1.5e-2),
}

plt_distr(df_descrs, plot_info)

#### Получим embeddings для описания

In [ ]:
import tqdm

import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Возьмем предобченную модель
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny").to(device)

In [ ]:
# разобъем на батчи для подачи в модель
descrs = np.array_split(df_descrs["description"].values, 60 * 4)
len(descrs), len(descrs[5])

In [ ]:
all_embeddings = []

for i in tqdm.tqdm(range(len(descrs))):

    sentences = descrs[i].tolist()

    encoded_input = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        max_length=124,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        model_output = model(**encoded_input)

    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings).to("cpu")

    all_embeddings.append(embeddings)

In [ ]:
# Сохраним в бинарник
with open(data_path + "descrs_embs.dill", "wb") as f:
    dill.dump(torch.cat(all_embeddings).numpy(), f)

In [ ]:
# Загрузим данные
with open(data_path + "descrs_embs.dill", "rb") as f:
    all_embeddings = dill.load(f)

In [ ]:
all_embeddings

### Снизим размерность

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
components_to_keep = 10

In [ ]:
pca_lowrank = PCA(n_components=components_to_keep)
all_embeddings = pca_lowrank.fit_transform(all_embeddings)
all_embeddings.shape

In [ ]:
txt_embs_df = pd.DataFrame(
    all_embeddings,
    columns=[f"txt_emb_pca_{i}" for i in range(components_to_keep)],
)
txt_embs_df

In [ ]:
df_descrs = pd.concat(
    [
        df_descrs.reset_index(drop=True),
        txt_embs_df.reset_index(drop=True),
    ],
    axis=1,
)

df_descrs

In [ ]:
# Сохраним в бинарник
with open(data_path + "df_descrs.dill", "wb") as f:
    dill.dump(df_descrs, f)

Названия товаров решено отбросить, т.к. по сути все товары являются платьями, 
и лишь в названиях небольшого количества товаров, содержится иформация дублируемая в описании 

## Brands

In [ ]:
df_brands = pd.DataFrame([], columns=["nm_id", "brand"])

for batch in df_text_pq.to_batches():

    tmp = batch.to_pandas()[["nm_id", "brandname"]]
    tmp["brandname"] = tmp["brandname"].astype(str).apply(str.lower)
    tmp = tmp.rename(
        columns={
            "brandname": "brand",
        }
    )

    df_brands = pd.concat([df_brands, tmp], axis=0)

df_brands

In [ ]:
# Сохраним в бинарник
with open(data_path + "df_brands.dill", "wb") as f:
    dill.dump(df_brands, f)

## Смерджим данные по айтемам

In [ ]:
with open(data_path + "df_descrs.dill", "rb") as f:
    df_descrs = dill.load(f)

with open(data_path + "df_brands.dill", "rb") as f:
    df_brands = dill.load(f)

with open(data_path + "items_colors.dill", "rb") as f:
    df_colors = dill.load(f)

with open(data_path + "chars_df.dill", "rb") as f:
    chars_df = dill.load(f)

In [ ]:
df_items = pd.merge(
    left=df_descrs,
    right=df_brands,
    left_on="nm_id",
    right_on="nm_id",
)
df_items = pd.merge(
    left=df_items,
    right=df_colors,
    left_on="nm_id",
    right_on="nm_id",
)
df_items = pd.merge(
    left=df_items.rename(
        columns={
            "nm_id": "item_id",
        }
    ),
    right=chars_df,
    left_on="item_id",
    right_on="item_id",
)
df_items

## Drop title & descr text data

In [ ]:
df_items = df_items.drop(columns=["title", "description"])
df_items["item_id"] = df_items["item_id"].astype(int)
df_items.head()

In [ ]:
# Сохраним в бинарник
with open(data_path + "df_items.dill", "wb") as f:
    dill.dump(df_items, f)